In [1]:
!nvidia-smi

Wed Sep  6 17:47:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
# Write the modified CUDA code to a file
%%writefile inner_product_with_testbench.cu

// Import required header files
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Kernel function for inner-product-based matrix multiplication
__global__ void innerProductKernel(float *A, float *B, float *C, int N) {
    // Calculate row and column indices for the thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Declare a variable to accumulate the sum
    float sum = 0;

    // Check if the thread's indices are within the matrix dimensions
    if(row < N && col < N) {
        // Compute the dot product for the i-th row of A and the j-th column of B
        for(int k = 0; k < N; ++k) {
            sum += A[row * N + k] * B[k * N + col];
        }
        // Store the sum in the corresponding element of matrix C
        C[row * N + col] = sum;
    }
}

// Function to test the inner product implementation
void testInnerProduct(float *A, float *B, float *C, int N) {
    // Declare device pointers for matrices A, B, and C
    float *d_A, *d_B, *d_C;

    // Error handling for CUDA operations
    cudaError_t err;

    // Allocate memory on the GPU for matrices A, B, and C
    err = cudaMalloc((void **)&d_A, N * N * sizeof(float));
    if (err != cudaSuccess) {
        printf("CUDA malloc A: %s\n", cudaGetErrorString(err));
        exit(1);
    }

    err = cudaMalloc((void **)&d_B, N * N * sizeof(float));
    if (err != cudaSuccess) {
        printf("CUDA malloc B: %s\n", cudaGetErrorString(err));
        exit(1);
    }

    err = cudaMalloc((void **)&d_C, N * N * sizeof(float));
    if (err != cudaSuccess) {
        printf("CUDA malloc C: %s\n", cudaGetErrorString(err));
        exit(1);
    }

    // Copy the data for matrices A and B from host to device
    cudaMemcpy(d_A, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * N * sizeof(float), cudaMemcpyHostToDevice);

    // Define the number of threads per block and the number of blocks per grid
    dim3 threadsPerBlock(2, 2);
    dim3 blocksPerGrid(N / threadsPerBlock.x, N / threadsPerBlock.y);

    // Launch the kernel
    innerProductKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Check for errors in kernel launch
    err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("Kernel launch error: %s\n", cudaGetErrorString(err));
        exit(1);
    }

    // Copy the resultant matrix C from device to host
    cudaMemcpy(C, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);

    // Free the allocated device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

// Main function
int main() {
    // Define the dimension of the matrices
    int N = 4;

    // Initialize matrices A and B
    float A[N * N] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16};
    float B[N * N] = {16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1};

    // Declare the result matrix C
    float C[N * N];

    // Run the test function
    testInnerProduct(A, B, C, N);

    // Print the resultant matrix C
    printf("Matrix C: \n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%f ", C[i * N + j]);
        }
        printf("\n");
    }

    // Exit the program
    return 0;
}

Overwriting inner_product_with_testbench.cu


In [18]:
# Compile the CUDA code with nvcc (NVIDIA CUDA Compiler)
!nvcc inner_product_with_testbench.cu -o inner_product_with_testbench

# Run the compiled CUDA program
!./inner_product_with_testbench

Matrix C: 
80.000000 70.000000 60.000000 50.000000 
240.000000 214.000000 188.000000 162.000000 
400.000000 358.000000 316.000000 274.000000 
560.000000 502.000000 444.000000 386.000000 


## CUDA Matrix Multiplication with Inner Product Approach

This notebook demonstrates how to perform matrix multiplication using the inner product approach on a GPU using CUDA. The code consists of two major parts:

### Kernel Function (`innerProductKernel`)

The `innerProductKernel` function is the GPU kernel that carries out the matrix multiplication. Each thread calculates one element of the resultant matrix \( C \) by taking the dot product of one row of matrix \( A \) with one column of matrix \( B \).

- `__global__ void innerProductKernel(float *A, float *B, float *C, int N)`: This is the CUDA kernel function where the actual multiplication takes place.

    - `float *A, *B, *C`: Pointers to matrices \( A \), \( B \), and \( C \) stored in the device memory.
  
    - `int N`: Dimension of the square matrices \( A \), \( B \), and \( C \).
    
    - `row` and `col`: Calculated to determine the row and column index for each thread.
    
    - `sum`: Variable to store the intermediate sum while calculating each element \( C_{ij} \).
  
### Test Function (`testInnerProduct`)

The `testInnerProduct` function serves as a testbench. It allocates device memory, copies the input matrices from host to device, launches the kernel, and then copies the resultant matrix back to the host.

- `void testInnerProduct(float *A, float *B, float *C, int N)`: Function that prepares the data, calls the CUDA kernel, and retrieves the data.

    - `float *d_A, *d_B, *d_C`: Device pointers for matrices \( A \), \( B \), and \( C \).
  
    - `cudaMalloc`: Allocates memory on the device.
    
    - `cudaMemcpy`: Transfers data between host and device.
    
    - `innerProductKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N)`: Kernel invocation.
    
    - `cudaFree`: Frees the allocated device memory.

### Compilation and Execution

The code is compiled using the NVIDIA CUDA Compiler (`nvcc`) and executed on the GPU. The resultant matrix \( C \) is then printed on the console.

To run this code:

1. Use the `%%writefile` magic command to write the CUDA code into a `.cu` file.
2. Use `!nvcc` to compile the code.
3. Run the compiled executable with `!./inner_product_with_testbench`.

This approach utilizes the parallel processing capabilities of a GPU to accelerate matrix multiplication.

In [11]:
# Write the CUDA code to a file
%%writefile outer_product_with_testbench.cu

// Import required header files
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Kernel function for outer-product-based matrix multiplication
__global__ void outerProductKernel(float *A, float *B, float *C, int N) {
    // Calculate row and column indices for the thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Declare a variable to accumulate the sum
    float sum = 0;

    // Check if the thread's indices are within the matrix dimensions
    if(row < N && col < N) {
        // Compute the dot product for the i-th row of A and the j-th column of B
        for(int k = 0; k < N; ++k) {
            sum += A[row * N + k] * B[k * N + col];
        }
        // Store the sum in the corresponding element of matrix C
        C[row * N + col] = sum;
    }
}

// Function to test the outer product implementation
void testOuterProduct(float *A, float *B, float *C, int N) {
    // Declare device pointers for matrices A, B, and C
    float *d_A, *d_B, *d_C;

    // Allocate memory on the GPU for matrices A, B, and C
    cudaMalloc((void **)&d_A, N * N * sizeof(float));
    cudaMalloc((void **)&d_B, N * N * sizeof(float));
    cudaMalloc((void **)&d_C, N * N * sizeof(float));

    // Copy the data for matrices A and B from host to device
    cudaMemcpy(d_A, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * N * sizeof(float), cudaMemcpyHostToDevice);

    // Define the number of threads per block and the number of blocks per grid
    dim3 threadsPerBlock(2, 2);
    dim3 blocksPerGrid(N / threadsPerBlock.x, N / threadsPerBlock.y);

    // Launch the kernel
    outerProductKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Copy the resultant matrix C from device to host
    cudaMemcpy(C, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);

    // Free the allocated device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

// Main function
int main() {
    // Define the dimension of the matrices
    int N = 4;
    // Initialize matrices A and B
    float A[N * N] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16};
    float B[N * N] = {16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1};
    // Declare the result matrix C
    float C[N * N];

    // Run the test function
    testOuterProduct(A, B, C, N);

    // Print the resultant matrix C
    printf("Matrix C: \n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%f ", C[i * N + j]);
        }
        printf("\n");
    }

    // Exit the program
    return 0;
}

Overwriting outer_product_with_testbench.cu


In [19]:
# Compile the CUDA code with nvcc (NVIDIA CUDA Compiler)
!nvcc outer_product_with_testbench.cu -o outer_product_with_testbench

# Run the compiled CUDA program
!./outer_product_with_testbench

Matrix C: 
80.000000 70.000000 60.000000 50.000000 
240.000000 214.000000 188.000000 162.000000 
400.000000 358.000000 316.000000 274.000000 
560.000000 502.000000 444.000000 386.000000 


## CUDA Matrix Multiplication with Outer Product Approach

This notebook demonstrates how to perform matrix multiplication using the outer product approach on a GPU using CUDA. The code consists of two major parts:

### Kernel Function (`outerProductKernel`)

The `outerProductKernel` function is the GPU kernel that performs the matrix multiplication in an outer-product-based approach. In this method, each thread is responsible for calculating an entire row-column multiplication, rather than a single element of the resultant matrix \( C \).

- `__global__ void outerProductKernel(float *A, float *B, float *C, int N)`: This is the CUDA kernel function where the actual multiplication takes place.

    - `float *A, *B, *C`: Pointers to matrices \( A \), \( B \), and \( C \) stored in the device memory.
  
    - `int N`: Dimension of the square matrices \( A \), \( B \), and \( C \).
    
    - `row` and `col`: Calculated to determine the row and column index for each thread.
    
    - `temp_C`: Variable to store the intermediate results of the outer product of two vectors for a specific element in \( C \).

### Test Function (`testOuterProduct`)

The `testOuterProduct` function serves as a testbench. It allocates device memory, copies the input matrices from host to device, launches the kernel, and then copies the resultant matrix back to the host.

- `void testOuterProduct(float *A, float *B, float *C, int N)`: Function that prepares the data, calls the CUDA kernel, and retrieves the data.

    - `float *d_A, *d_B, *d_C`: Device pointers for matrices \( A \), \( B \), and \( C \).
  
    - `cudaMalloc`: Allocates memory on the device.
    
    - `cudaMemcpy`: Transfers data between host and device.
    
    - `outerProductKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N)`: Kernel invocation.
    
    - `cudaFree`: Frees the allocated device memory.

### Compilation and Execution

The code is compiled using the NVIDIA CUDA Compiler (`nvcc`) and executed on the GPU. The resultant matrix \( C \) is then printed on the console.

To run this code:

1. Use the `%%writefile` magic command to write the CUDA code into a `.cu` file.
2. Use `!nvcc` to compile the code.
3. Run the compiled executable with `!./outer_product_with_testbench`.

This approach leverages the parallel processing power of a GPU to execute the outer-product-based matrix multiplication efficiently.